In [2]:
import re
import pandas as pd
from IPython.display import display

# Function to check if a sentence is specific or vague
def is_specific(sentence):
    score = 0
    # Check for clues that indicate specificity (numbers, dates, project names)
    if re.search(r'\d+%|\d+ percent|\d+\b', sentence): score += 2
    if re.search(r'on \w+ \d{1,2}(st|nd|rd|th)|during Q[1-4]|in \w+ 20\d{2}', sentence): score += 2
    if re.search(r'project [A-Z][a-z]+', sentence, re.IGNORECASE): score += 1
    # Look for "for example" and see if it is followed by specific info
    if "for example" in sentence.lower():
        example_text = sentence.lower().split("for example")[-1]
        if re.search(r'\d+|project|successfully|delivered|implemented', example_text):
            score += 1
        else:
            score -= 1
    # Penalize for vague patterns/phrases
    vague_phrases = [
        "good job", "great work", "excellent performance", "well done",
        "needs improvement", "work harder", "communication skills",
        "meets expectations", "in the future", "attention to detail"
    ]
    for vp in vague_phrases:
        if vp in sentence.lower():
            score -= 2
    # Return label based on score
    return "Specific" if score >= 1 else "Vague"

# Example feedback text, split into sentences
sample_text = """
John did a great job on the project.
The team successfully delivered Project X on October 26th, 2023, increasing efficiency by 15%.
The report was good, but some areas need improvement.
For example, the data analysis section could be enhanced further.
He needs to improve his communication skills in the future.
Attention to detail is important.
"""

# Split the text into individual sentences
sentences = [s.strip() for s in sample_text.strip().split('\n') if s.strip()]

rows = []
# Check specificity for each sentence
for sentence in sentences:
    spec = is_specific(sentence)
    rows.append({"Sentence": sentence, "Specificity": spec})

df = pd.DataFrame(rows)

# Function to color rows by specificity (green = specific, red = vague)
def color_row(row):
    color = "#c8e6c9" if row.Specificity == "Specific" else "#ffcdd2"
    return [f'background-color: {color}']*2

display(df.style.apply(color_row, axis=1))

# Suggestions for vague sentences
suggestions_map = {
    "good job": "Specify what was good or give an example.",
    "needs improvement": "Explain which areas need improvement and how.",
    "communication skills": "Give an example of a communication situation.",
    "in the future": "Be concrete about what should happen and when.",
    "attention to detail": "Describe a case where detail mattered.",
    "was good": "Explain why it was good and what could improve.",
    "could be enhanced": "State how it could be enhanced and the expected impact.",
}

print("🟠 Recommendations for vague sentences:")
for i, row in df[df.Specificity == "Vague"].iterrows():
    found = False
    for k, v in suggestions_map.items():
        if k in row.Sentence.lower():
            print(f"- \"{row.Sentence}\" ➡️ {v}")
            found = True
            break
    if not found:
        print(f"- \"{row.Sentence}\" ➡️ Try to add concrete details, numbers, or results.")


,Sentence,Specificity
0,John did a great job on the project.,Vague
1,"The team successfully delivered Project X on October 26th, 2023, increasing efficiency by 15%.",Specific
2,"The report was good, but some areas need improvement.",Vague
3,"For example, the data analysis section could be enhanced further.",Vague
4,He needs to improve his communication skills in the future.,Vague
5,Attention to detail is important.,Vague


🟠 Recommendations for vague sentences:
- "John did a great job on the project." ➡️ Try to add concrete details, numbers, or results.
- "The report was good, but some areas need improvement." ➡️ Explain why it was good and what could improve.
- "For example, the data analysis section could be enhanced further." ➡️ State how it could be enhanced and the expected impact.
- "He needs to improve his communication skills in the future." ➡️ Give an example of a communication situation.
- "Attention to detail is important." ➡️ Describe a case where detail mattered.
